<a href="https://colab.research.google.com/github/osvaldoherrera715-cyber/Vehicle-Accident-Prevention/blob/main/Dashcamv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
%pip install "ultralytics<=8.3.40" supervision roboflow
# prevent ultralytics from tracking your activity
!yolo settings sync=False
import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.7/112.6 GB disk)


In [4]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="omsEY0EeQVA60NK7nabV")
project = rf.workspace("osvaldos-work").project("car-accident-prevention-yolov11-ooxaw")
version = project.version(5)
dataset = version.download("yolov11")





loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Car-Accident-Prevention-Yolov11-5 in yolov11:: 100%|██████████| 544/544 [00:00<00:00, 4053.89it/s]


In [5]:
dataset = version.download("yolov11")
dataset_path = dataset.location


In [6]:
import os

print("📂 Dataset structure:\n")
for split in ['train', 'valid', 'test']:
    images_path = os.path.join(dataset_path, split, 'images')
    labels_path = os.path.join(dataset_path, split, 'labels')

    num_images = len(os.listdir(images_path)) if os.path.exists(images_path) else 0
    num_labels = len(os.listdir(labels_path)) if os.path.exists(labels_path) else 0

    print(f"{split.upper()}:")
    print(f"  Images: {num_images}")
    print(f"  Labels: {num_labels}\n")




📂 Dataset structure:

TRAIN:
  Images: 234
  Labels: 234

VALID:
  Images: 22
  Labels: 22

TEST:
  Images: 10
  Labels: 10



In [7]:
from ultralytics import YOLO

# Load the YOLOv11n model (lightweight)
model = YOLO('yolo11n.pt')


100%|██████████| 5.35M/5.35M [00:00<00:00, 104MB/s]


In [ ]:
# Train
model.train(
   data=f"{dataset.location}/data.yaml",  # your dataset path
    epochs=50,        # enough to converge
    imgsz=1280,       # downscale to fit GPU memory
    batch=8,          # safe for Colab T4
    device=0          # use GPU
)


New https://pypi.org/project/ultralytics/8.3.217 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/Car-Accident-Prevention-Yolov11-5/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=1280, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, 

100%|██████████| 755k/755k [00:00<00:00, 28.9MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/Car-Accident-Prevention-Yolov11-5/train/labels... 234 images, 0 backgrounds, 0 corrupt: 100%|██████████| 234/234 [00:00<00:00, 2464.46it/s]

train: New cache created: /content/Car-Accident-Prevention-Yolov11-5/train/labels.cache


...

      49/50      4.92G     0.8051     0.7577     0.8888          6       1280: 100%|██████████| 30/30 [00:13<00:00,  2.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.02it/s]


                   all         22        102      0.825      0.877      0.878      0.568

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      4.93G     0.8062     0.7733     0.8763          4       1280: 100%|██████████| 30/30 [00:13<00:00,  2.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.45it/s]

                   all         22        102      0.822      0.872      0.865      0.552



50 epochs completed in 0.237 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


In [10]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/runs/detect/train/weights/best.pt /content/drive/MyDrive/


Mounted at /content/drive
